In [90]:
#Import the package needed
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import csv
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")

In [91]:
data = pd.read_csv('./data/datasets/raw.csv')

# clean_train_data, clean_test_data = train_test_split(data_clean, test_size=0.2, random_state=42)

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

print("Train data : {}, Val Data: {}, Test Data: {}".format(train_data.shape[0], val_data.shape[0], test_data.shape[0]))

Train data : 54058, Val Data: 18020, Test Data: 18020


In [92]:
# Label encode the "Category" and "Style" columns in the train_data, val_data, and test_data dataframes

from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the label encoder on the "Category" column in the train_data dataframe
label_encoder.fit(train_data['Category'])

# Encode the "Category" column in the train_data dataframe
train_data['Category'] = label_encoder.transform(train_data['Category'])

# Encode the "Category" column in the val_data dataframe
val_data['Category'] = label_encoder.transform(val_data['Category'])

# Encode the "Category" column in the test_data dataframe
test_data['Category'] = label_encoder.transform(test_data['Category'])



# Fit the label encoder on the "Style" column in the train_data dataframe
label_encoder.fit(train_data['Style'])

# Encode the "Style" column in the train_data dataframe
train_data['Style'] = label_encoder.transform(train_data['Style'])

# Encode the "Style" column in the val_data dataframe
val_data['Style'] = label_encoder.transform(val_data['Style'])

# Encode the "Style" column in the test_data dataframe
test_data['Style'] = label_encoder.transform(test_data['Style'])




In [93]:
# train_data = train_data.drop("Path", axis=1)
# test_data = test_data.drop("Path", axis=1)
#  val_data = val_data.drop("Path", axis=1)

In [94]:
train_data['Category'] = train_data['Category'].astype('str')
val_data['Category'] = val_data['Category'].astype('str')

In [95]:
train_data

,Image_Name,Category,Style,Category_Path,Full_Path
74633,1583contemporary-side-tables-and-end-tables.jpg,6,2,./data/datasets/raw/tables/Contemporary,./data/datasets/raw/tables/Contemporary/1583co...
62545,19476transitional-table-lamps.jpg,4,15,./data/datasets/raw/lamps/Transitional,./data/datasets/raw/lamps/Transitional/19476tr...
83052,44235midcentury-side-tables-and-end-tables.jpg,6,9,./data/datasets/raw/tables/Midcentury,./data/datasets/raw/tables/Midcentury/44235mid...
61685,8174traditional-table-lamps.jpg,4,14,./data/datasets/raw/lamps/Traditional,./data/datasets/raw/lamps/Traditional/8174trad...
47877,27647craftsman-table-lamps.jpg,4,3,./data/datasets/raw/lamps/Craftsman,./data/datasets/raw/lamps/Craftsman/27647craft...
...,...,...,...,...,...
19500,5143southwestern-armchairs-and-accent-chairs.jpg,2,13,./data/datasets/raw/chairs/Southwestern,./data/datasets/raw/chairs/Southwestern/5143so...
3601,6058traditional-panel-beds.jpg,1,14,./data/datasets/raw/beds/Traditional,./data/datasets/raw/beds/Traditional/6058tradi...
44069,3430contemporary-floor-lamps.jpg,4,2,./data/datasets/raw/lamps/Contemporary,./data/datasets/raw/lamps/Contemporary/3430con...
18177,6940modern-dining-chairs.jpg,2,10,./data/datasets/raw/chairs/Modern,./data/datasets/raw/chairs/Modern/6940modern-d...


In [96]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='./',
        x_col="Full_Path",
        y_col="Category",
        target_size=(350, 350),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='./',
        x_col="Full_Path",
        y_col="Category",
        target_size=(350, 350),
        batch_size=batch_size,
        class_mode='categorical')

Found 54047 validated image filenames belonging to 6 classes.
Found 18019 validated image filenames belonging to 6 classes.


In [97]:
def plot_learning_curve(train_loss, val_loss, train_metric, val_metric, metric_name='Accuracy'):
    plt.figure(figsize=(10,5))
    
    plt.subplot(1,2,1)
    plt.plot(train_loss, 'r--')
    plt.plot(val_loss, 'b--')
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.legend(['train', 'val'], loc='upper left')

    plt.subplot(1,2,2)
    plt.plot(train_metric, 'r--')
    plt.plot(val_metric, 'b--')
    plt.xlabel("epochs")
    plt.ylabel(metric_name)
    plt.legend(['train', 'val'], loc='upper left')

    plt.show()

In [98]:
INPUT_DIM = (350,350,3)
HIDDEN_LAYER_DIM = 256
OUTPUT_CLASSES = 6

In [99]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam

In [100]:
model = Sequential([
    Flatten(input_shape=INPUT_DIM),
    Dense(HIDDEN_LAYER_DIM, activation='relu'),
    Dense(OUTPUT_CLASSES, activation='sigmoid')
])

In [101]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 367500)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │    94,080,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,082,312 (358.90 MB)

 Trainable params: 94,082,312 (358.90 MB)

 Non-trainable params: 0 (0.00 B)

In [102]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [103]:
model.compile(optimizer='SGD',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['categorical_accuracy'])

In [104]:
history = model.fit(train_generator, epochs=50, validation_data=validation_generator, verbose=0)

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 6), output.shape=(None, 8)